# Brownian Motion Simulations

**"My functions"** have the functions I wrote myself. Below, **"Package functions"** have the functions from various R packages. These packages are: "somebm" "fExpressCertificates" and "yuima". The first two give the same results as my functions. "yuima" differs slightly most of the time and considerably for certain parameter values. I specified them below. After that, I gathered all functions under a heading for a quick comparison of results. round(vector, n) == round(vector, n) gives "TRUE" up to 4 decimal points. The last heading "Visualizations" involves a function for plotting. I made it a function because it's easier to use that way. Make sure that when you enter the output of a BM function "x" as the input of the plotting function, you index it as x[[1]]. The output is a list with two elements. The first is a matrix of BM values, the second is a vector of hitting times. This does not apply to standard Brownian motion because it does not have an absorbing barrier. 

## My functions 

## Standard Brownian Motion
This function is actually not needed, first because it does not have a drift. Also, the ABM function does the same thing with mu = 0, provided we set the absorbing barrier L sufficiently above or below 0. This is because in the standard BM, the starting value is always 0. Consequently, the process may hit L - despite not having drift - and set the remaining values to NA. This will not be a problem in ABM and GBM, because we will start with a value above 0). 

In [0]:
my_bm <- function(t, n, nsim){ # t is the terminal/final time point
                               # n is the number of intervals between 0:t
                               # nsim is the number of sample paths
  sig2 <- t/n # the variance of each increment equals its length 
  time <- seq(from = 0, to = t, by = sig2) # time vector with t/sig2 + 1 = n + 1 elements 
  sigma <- sqrt(sig2) # the standard deviation of each increment 
  ri <- rnorm(n = nsim * (length(time) - 1), mean = 0, sd = sigma) # simulating 100 normally distributed increments for all simulations
  X <- matrix(data = ri, nrow = nsim, ncol = length(time) - 1) # storing these increments in a i(number of simulations) x j(number of increments in a simulation) matrix 
  X <- cbind(rep(0, nsim), t(apply(X, 1, cumsum))) # combining nsim 0s for the starting value of each BM and
  return(X)                                        # the new values of the BM as the cumulative sum of the value of the BM at each time point
}
set.seed(1)
bm1 <- my_bm(t = 1, n = 100, nsim = 1) 
bm1

0,0.04608108,0.1803913,0.2801081,0.2433457,0.2301474,0.4078574,0.3614305,0.3440458,0.3036444,⋯,-0.4973242,-0.5286845,-0.5078405,-0.6278036,-0.6512623,-0.8260704,-0.8182794,-0.8276161,-0.8244461,-0.7079515


In [0]:
bmplot(x, nsim, t, n, ylim, title) # the function is below  

ERROR: ignored

## Arithmetic Brownian Motion (with drift and absorbing parameter)

In [0]:
my_abm <- function(nsim, t, n, X0, mu, sigma, L){ # nsim is the number of BM sample paths to be simulated 
                                                  # t is the final time point 
                                                  # n is the number of time intervals/increments from 0:t
                                                  # X0 is the first value of BM at time 0, mu is the drift 
                                                  # sigma is the diffusion coefficient and L is the barrier
  
  dt <- t/n # time divided into equal intervals; change in time and also the variance of the increments
  sig2 <- sigma^2 # the volatility term by which the process is scaled, not the variance of the increments 
  time <- seq(from = 0, to = t, by = dt) # a time vector that determines the number of columns, with rows being the simulated sample paths  
  
  initial <- X0 # the initial value of each simulated sample path 
  X <- matrix(nrow = nsim, ncol = length(time)) # the matrix that will store all the values
  X[1:nsim, 1] <- X0 # setting the first value of each simulation 
  hit_times <- numeric(length = nsim)
  
  for(i in 1:nrow(X)){
    for(j in 2:length(time)){
      X[i,j] <- X0 + mu * dt + sigma * sqrt(dt) * rnorm(n = 1, mean = 0, sd = 1) # the formula for the next value of BM
      if(X[i,j] > L & j < ncol(X)){ # if the value is larger than the absorbing barrier L, meaning we have not hit it yet, and if the time point (ncol) is not the last time point of a given simulation,
        X0 <- X[i,j] # then the initial value of BM is updated by the new value 
      } else if(X[i,j] > L & j == ncol(X)){ # if we have not hit L but we have come to the last time point in the simulation,
        X0 <- initial # then the initial value is updated by 1, which is the first value of every simulation
        hit_times[i] <- length(time) # and the hit time is none, so I set it as the last time point
      } else if(X[i,j] <= L){ # if we have hit the absorbing barrier from above in a given simulation
        X0 <- initial # then the initial value is updated by 1, since we will start the new BM, and
        hit_times[i] <- j # the time at which we reach L (the column number) updates the hitting time vector
        break
      }
    }
    values <- list(Values = X, Hittings = hit_times)
  }
  return(values)
}

set.seed(1)
abm1 <- my_abm(nsim = 1, t = 1, n = 100, X0 = 1, mu = -1, sigma = 1, L = 0)
abm1

1,1.036081,1.160391,1.250108,1.203346,1.180147,1.347857,1.29143,1.264046,1.213644,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [0]:
bmplot(x, nsim, t, n, ylim, title) # the function is below

ERROR: ignored

A data frame that includes the hitting time of each simulated BM


In [0]:
failures <- data.frame(Simulations = c(paste(rep("Sim", 100), c(1:100), "")), Hitting_times = x1$`Hitting times`) 
hist(failures$Hitting_times, xlab = "Hitting times", ylab = "Frequency", 
     main = "Hitting times of 1000 GBM sample paths", xlim = c(0, 1000))

ERROR: ignored

## Geometric Brownian Motion (with absorbing barrier)
The only difference between this and the ABM function is the formula. The variables are the same.

In [0]:
my_gbm <- function(nsim, t, n, X0, mu, sigma, L){ # nsim is the number of BM sample paths to be simulated 
                                                  # t is the final time point 
                                                  # n is the number of time intervals/increments from 0:t
                                                  # X0 is the first value of BM at time 0, mu is the drift 
                                                  # sigma is the diffusion coefficient, L is the barrier
  
  dt <- t/n # time divided into equal intervals; change in time and also the variance of the increments
  sig2 <- sigma^2 # the volatility term by which the process is scaled, not the variance of the increments 
  time <- seq(from = 0, to = t, by = dt) # a time vector that determines the number of columns, with rows being the simulated sample paths  
  
  initial <- X0 # the initial value of each simulated sample path 
  X <- matrix(nrow = nsim, ncol = length(time)) # the matrix that will store all the values
  X[1:nsim, 1] <- X0 # setting the first value of each simulation 
  hit_times <- numeric(length = nsim) # one hitting time for each simulated sample path  
  
  for(i in 1:nrow(X)){
    for(j in 2:length(time)){
      X[i,j] <- X0 * exp(((mu - 0.5 * sig2) * dt) + (sigma * sqrt(dt) * rnorm(n = 1, mean = 0, sd = 1))) # the formula for the next value of BM
      if(X[i,j] > L & j < ncol(X)){ # if the value is larger than the absorbing barrier L, meaning we have not hit it yet, and if the time point (ncol) is not the last time point of a given simulation,
        X0 <- X[i,j] # then the initial value of BM is updated by the new value 
      } else if(X[i,j] > L & j == ncol(X)){ # if we have not hit L but we have come to the last time point in the simulation,
        X0 <- initial # then the initial value is updated by 1, which is the first value of every simulation
        hit_times[i] <- length(time) # and the hit time is none
      } else if(X[i,j] <= L){ # if we have hit the absorbing barrier from above in a given simulation
        X0 <- initial # then the initial value is updated by 1, since we will start the new BM, and
        hit_times[i] <- j # the time at which we reach L (the column number) updates the hitting time vector
        break
      }
    }
    values <- list("Values" = X, "Hitting times" = hit_times)
  }
  return(values)
}

set.seed(1)
gbm1 <- my_gbm(nsim = 1, t = 1, n = 100, X0 = 1, mu = -1, sigma = 1, L = 0.1)
gbm1

In [0]:
bmplot(x, nsim, t, n, ylim, title) # the function is below

The same data frame for the hitting times 

In [0]:
failures <- data.frame(Simulations = c(paste(rep("Sim", 100), c(1:100), "")), Hitting_times = x1$`Hitting times`) 
hist(failures$Hitting_times, xlab = "Hitting times", ylab = "Frequency", 
     main = "Hitting times of 1000 GBM sample paths", xlim = c(0, 1000))

ERROR: ignored

# Package functions 

## The "fExpressCertificates" package 

In [0]:
#install.packages("fExpressCertificates")
library(fExpressCertificates)  
set.seed(1)
bm2 <- fExpressCertificates::BM(S0 = 0, mu = 0, sigma = 1, T = 1, N = 100) # standard 
abm2 <- fExpressCertificates::BM(S0 = 1, mu = -1, sigma = 1, T = 1, N = 100) # arithmetic 
gbm2 <- fExpressCertificates::GBM(S0 = 1, mu = -1, sigma = 1, T = 1, N = 100) # geometric 

## The "somebm" package (just standard and geometric)


In [0]:
#install.packages("somebm")
library(somebm)
set.seed(1)
bm3 <- somebm::bm(x0 = 0, t0 = 0, t = 1, n = 100) # standard 
gbm3 <- somebm::gbm(x0 = 1, mu = -1, sigma = 1, t0 = 0, t = 1, n = 100) # geometric 


## GBM with the yuima package
This mostly gives the same results, except when "Terminal"/the last point is much larger (e.g., 100 instead of 1 for all functions). This is not a problem with the other packages.

In [0]:
install.packages("yuima")
require(yuima)
set.seed(1)
m <- setModel(drift = "mu*s", diffusion = "sigma*s", state.var = "s", time.var = "t", solve.var = "s", xinit = 1)
options(warn = -1)# Suppress Warnings from R
simnum <- 1 # number of BMs to be simulated 
grid <- setSampling(Initial = 0, Terminal = 1, n = 100) # Initial is the first time point, Terminal is the last time point, n is the number of intervals 
newsim <- function(i){simulate(m, sampling = grid, true.param = list(mu = -1, sigma = 1))@data@original.data}
options(warn = -1)# Suppress Warnings from R
sim <- sapply(1:simnum, function(x)newsim(x))
gbm4 <- t(sim) # this is the final matrix with the values of all the simulations 

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



This is independent from the package. I added it to get the hitting times

In [0]:
hittings <- numeric(length = nrow(x3)) 
for(i in 1:nrow(x3)){
  for(j in 2:ncol(x3)){
    if(x3[i,j] <= 0.001){
      hittings[i] <- j
      break
    }
  }
}
hist(hittings)

ERROR: ignored

# Comparing all four functions

## Standard Brownian motion

In [0]:
set.seed(1)
bm1 <- my_bm(t = 1, n = 100, nsim = 1) 
bm2 <- my_abm(nsim = 1, t = 1, n = 100, X0 = 0, mu = 0, sigma = 1, L = -100) # my abm with mu = 0
bm3 <- fExpressCertificates::BM(S0 = 0, mu = 0, sigma = 1, T = 1, N = 100) # other abm with mu = 0 
bm4 <- somebm::bm(x0 = 0, t0 = 0, t = 1, n = 100)


In [0]:
bm1[1:10]
bm2[1, 1:10]
bm3[1:10]
bm4[1:10]

## Arithmetic Brownian motion with drift


In [0]:
set.seed(1)
abm1 <- my_abm(nsim = 1, t = 1, n = 100, X0 = 0, mu = -1, sigma = 1, L = -100)
abm2 <- fExpressCertificates::BM(S0 = 0, mu = -1, sigma = 1, T = 1, N = 100)


In [0]:
abm1[1, 1:10]
abm2[1:10]

## Geometric Brownian motion
All except the yuima package give the same result (and yuima is very similar)


In [0]:
set.seed(1)
gbm1 <- my_gbm(nsim = 2, t = 100, n = 100, X0 = 1, mu = -1, sigma = 1, L = -100) # my gbm 
gbm2 <- fExpressCertificates::GBM(S0 = 1, mu = -1, sigma = 1, T = 100, N = 100)
gbm3 <- somebm::gbm(x0 = 1, mu = -1, sigma = 1, t0 = 0, t = 1, n = 100)


In [0]:
gbm1[[1]][1, 1:10]
gbm2[1:10]
gbm3[1:10]
gbm4[2, 1:10] # the yuima package 

# Visualizations

In [0]:
library(ggplot2)
library(reshape2)
bmplot <- function(x, nsim, t, n, L, ylim, title){ # x is the matrix output of the BM functions, n is the number of simulations, t is the vector of time points, as in the BM functions 
  rownames(x) <- paste("sim", seq(nsim), sep = "") # the number of simulations / rows
  colnames(x) <- paste("time", seq(0:n), sep = "") # the number of time points - 0:100 at the moment / columns 
  dat <- as.data.frame(x) # creating the data frame for ggplot 
  dat$sim <- rownames(dat)
  mdat <- melt(dat, id.vars = "sim")
  mdat$time <- as.numeric(gsub("time", "", mdat$variable))
  
  p <- ggplot(data = mdat, mapping = aes(x = time, y = value, group = sim)) +
  theme_bw() +
  theme(panel.grid = element_blank(), 
        plot.title = element_text(hjust = 0.5),
        axis.title.y = element_text(angle = 0, size = 11, margin = margin(t = 0, r = 10, b = 0, l = 0)), 
        axis.title.x = element_text(margin = margin(t = 10, b = 10))) +
  geom_line(size = 0.3, alpha = 1, aes(color = sim), show.legend = FALSE) + 
    ggtitle(title) + xlab("Time") + ylab("Value") + ylim(ylim) + geom_hline(yintercept = L, color = "orange", size = 0.3)
  return(p)
}

# Simulations 

### Storing the values: These two functions will allow us to nealty store the resutls. It is easier to define them before running the simulations. I wrote them because the results of parallel simulations are a mess. 


### The BM values 

In [0]:
values <- function(x, nsim, n){
  u <- unlist(x)
  u <- u[-which(names(u) == "Hittings")]
  m <- matrix(data = u, nrow = nsim, ncol = length(0:n), byrow = TRUE)
  df <- data.frame(x = t(m), row.names = paste0("Time", 0:n, ""))
  colnames(x = df) <- paste0("Sim", 1:nsim, "")
  store <- list(m, df)
  return(store)
}

### The hitting times 

In [0]:
hittings <- function(x, nsim, n){
  u <- unlist(x)
  u <- u[-which(names(u) != "Hittings")]
  m <- matrix(data = u, nrow = nsim, ncol = 1, byrow = TRUE)
  df <- data.frame(x = m, row.names = paste0("Sim", 1:nsim, ""))
  colnames(x = df) <- "Hitting time"
  store <- list(m, df)
  return(store)
}


### Parallel processing function 

In [0]:
install.packages("parallel")
library(parallel)
RNGkind("L'Ecuyer-CMRG") 
detectCores()

In [0]:
f <- function(i){ # specify the desired function and parameter values here
  my_abm(nsim = 1, t = 1, n = 100, X0 = 1, mu = -1, sigma = -0.15, L = 0) # keep nsim = 1 for now, will change later
}

In [0]:
set.seed(1)
res <- mclapply(X = 1:100, f, mc.cores = 8, mc.set.seed = TRUE) # X is the n of sim as a vector 
                                                                 # f is the function defined above 
                                                                 # n of cores you want to use 

In [0]:
s <- values(x = res, nsim = 100, n = 100) # indexing the BM values 
mval <- s[[1]] # BM values in a matrix (goes into the plotting function)
dfval <- s[[2]] # BM values in a data frame

h <- hittings(x = res, nsim = 100, n = 100) # indexing the hitting times 
mhit <- h[[1]] # in a matrix (for histograms)
dfhit <- h[[2]] # in a data frame 

In [0]:
p <- bmplot(x = mval, nsim = 100, t = 1, n = 100, L = 0, ylim = c(min(mval), max(mval)), # Define the range of the y-axis 
title = "Arithmetic Brownian motion with negative drift and an absorbing barrier")

In [0]:
print(p)

In [0]:
hist(mhit) # Histogram of hitting times 